# IBM Capstone Project

ZhichengZhu

# Introduction

## 1.1 Business Problem
New York is a city located in New York State, USA, and is the most populous city in the United States. In this project, we gonna choose this city as an example to use data analyst skill I learnt <br>
in the IBM data science course.<br>
We intend to find the best place to open an Chinese resturnat in New York. The target audience could be the one who want to find a place to open the Chinese resturant in New York. We gonna find <br>
the place without too much Chinese resturant and also with enough people lives. To use k-means clustering and foursquare to analyze data and create maps and also use panda to import data. 
## 1.2Methodology
The purpose of the analysis is to highlight the best Chinese restaurant locations in order to provide entrepreneurs with valuable suggestions. To this end, we will use the machine learning K-Means algorithm to divide New York into several clusters and give a "scoring" form for developing restaurant business from the most interesting area to the least interesting area. But before that, we will first explore the downtown area of New York to get all the restaurant information (data) in the downtown area. This will help us easily highlight the most visited food districts.
# 2 Data
## 2.1 Data Collection
All geographic coordinates of Queens, New York will be downloaded here: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json we will Create a table in which each region is listed as a row, and the longitude and latitude are listed as columns.

In the next step, we collect the locations of each department (listed in the table above) and look at the most common locations. In this step, we will use the Foursquare API to collect the name, category, GPS coordinates of all the data about the place. After collecting the data and sorting it into a panda data frame, we will get a table listing the top 10 locations for each department.

The final task is to use unsupervised machine learning techniques to cluster them according to the most common places, and use the Folio library to visualize all clusers.

# Import Data and third-party Libraries

In [25]:
import pandas as pd
!pip install geopandas
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip install folium
!pip install geopy
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install wikipedia
import wikipedia as wp
import requests
!pip install geojson
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopandas as gpd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import folium # map rendering library
from folium import plugins
from folium.plugins import HeatMap
from sklearn.cluster import KMeans
import urllib.request

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
with open('newyork_data.json') as json_data:
    ny_data = json.load(json_data)    
print('Data downloaded!')

Data downloaded!


In [3]:
neighborhoods_data = ny_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [4]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [5]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [6]:
import folium
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Borough'], queens_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [7]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
           
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)
            
            response = requests.get(url).json()
            results = response["response"]['venues']

            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [9]:
LIMIT = 400 
radius = 4500 
CLIENT_ID = '22A1FV0L4R2FD4BUGXIPAQTSVUY0KEHZVLJM3TUKK3HZNYHF' # your Foursquare ID
CLIENT_SECRET = 'IPR4XZRHZ5DSQEGUI1OQABZXVFVBEGP2ZQLPXKXO5WRACQGL' # your Foursquare Secret
ACCESS_TOKEN = '0MTNXRYHGSOEFROFDZ1BS14TYQYEJANK3TYZ5M3RMLQUDYE2' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 22A1FV0L4R2FD4BUGXIPAQTSVUY0KEHZVLJM3TUKK3HZNYHF
CLIENT_SECRET:IPR4XZRHZ5DSQEGUI1OQABZXVFVBEGP2ZQLPXKXO5WRACQGL


In [10]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_venues_chr = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d145941735')
queens_venues_chr.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Golden Kitchen,40.770366,-73.919031,Chinese Restaurant
1,Astoria,40.768509,-73.915654,Golden House Chinese Restaurant,40.765193,-73.917760,Chinese Restaurant
2,Astoria,40.768509,-73.915654,Sampan,40.764319,-73.914666,Chinese Restaurant
3,Astoria,40.768509,-73.915654,Ma Lala,40.762983,-73.913353,Chinese Restaurant
4,Astoria,40.768509,-73.915654,Flower Drum House,40.777767,-73.908625,Chinese Restaurant


In [11]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)
map_chr = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(queens_venues_chr, 'red', map_chr)

map_chr

In [12]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0
queens_grouped = queens_venues_chr.groupby('Neighborhood').count()
queens_grouped.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,2,2,2,2,2,2
Astoria,30,30,30,30,30,30
Astoria Heights,4,4,4,4,4,4
Auburndale,12,12,12,12,12,12
Bay Terrace,1,1,1,1,1,1


# Neighborhood Battle 

In [13]:
queens_onehot = pd.get_dummies(queens_venues_chr[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues_chr['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant,Dumpling Restaurant,Food Stand,Fried Chicken Joint,Gluten-free Restaurant,Halal Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Indian Restaurant,Japanese Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Noodle House,Seafood Restaurant,Shanghai Restaurant,Spanish Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Astoria,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Astoria,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Astoria,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Astoria,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Astoria,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped.head()

,Neighborhood,Asian Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant,Dumpling Restaurant,Food Stand,Fried Chicken Joint,Gluten-free Restaurant,Halal Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Indian Restaurant,Japanese Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Noodle House,Seafood Restaurant,Shanghai Restaurant,Spanish Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Arverne,0.000000,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Astoria,0.000000,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Astoria Heights,0.000000,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Auburndale,0.083333,0.0,0.0,0.083333,0.0,0.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bay Terrace,0.000000,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
1,Astoria,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
2,Astoria Heights,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
3,Auburndale,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Halal Restaurant,Bakery,Buffet,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
4,Bay Terrace,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant


# K-means Clustering

In [16]:
kclusters = 5
queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)
kmeans.labels_[0:10]


array([0, 0, 0, 4, 0, 2, 0, 0, 0, 0], dtype=int32)

In [17]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

queens_merged = queens_data
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,0.0,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
1,Queens,Woodside,40.746349,-73.901842,4.0,Chinese Restaurant,Halal Restaurant,Indian Restaurant,Vietnamese Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant
2,Queens,Jackson Heights,40.751981,-73.882821,2.0,Chinese Restaurant,Taiwanese Restaurant,Spanish Restaurant,Seafood Restaurant,Cantonese Restaurant,Indian Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Bakery,Buffet
3,Queens,Elmhurst,40.744049,-73.881656,2.0,Chinese Restaurant,Taiwanese Restaurant,Indian Restaurant,Shanghai Restaurant,Seafood Restaurant,Cantonese Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Bakery,Buffet
4,Queens,Howard Beach,40.654225,-73.838138,0.0,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant


In [19]:
queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
4,Howard Beach,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
5,Corona,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
7,Kew Gardens,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant
10,Long Island City,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
11,Sunnyside,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
12,East Elmhurst,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
13,Maspeth,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
15,Glendale,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
17,Woodhaven,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant


In [20]:
queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Jackson Heights,Chinese Restaurant,Taiwanese Restaurant,Spanish Restaurant,Seafood Restaurant,Cantonese Restaurant,Indian Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Bakery,Buffet
3,Elmhurst,Chinese Restaurant,Taiwanese Restaurant,Indian Restaurant,Shanghai Restaurant,Seafood Restaurant,Cantonese Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Bakery,Buffet
6,Forest Hills,Chinese Restaurant,Shanghai Restaurant,Szechuan Restaurant,Kosher Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Bakery,Buffet,Cantonese Restaurant
16,Rego Park,Chinese Restaurant,Shanghai Restaurant,Taiwanese Restaurant,Kosher Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Bakery,Buffet,Cantonese Restaurant
20,College Point,Chinese Restaurant,Taiwanese Restaurant,Buffet,Vietnamese Restaurant,Halal Restaurant,Bakery,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant
22,Bayside,Chinese Restaurant,Taiwanese Restaurant,Shanghai Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop
28,Kew Gardens Hills,Chinese Restaurant,Asian Restaurant,Indian Chinese Restaurant,Gluten-free Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop
29,Fresh Meadows,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Halal Restaurant,Bakery,Buffet,Caribbean Restaurant,Dim Sum Restaurant
32,Oakland Gardens,Chinese Restaurant,Vietnamese Restaurant,Taiwanese Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant
51,Murray Hill,Chinese Restaurant,Taiwanese Restaurant,Szechuan Restaurant,Asian Restaurant,Shanghai Restaurant,Korean Restaurant,Fried Chicken Joint,Bakery,Buffet,Cantonese Restaurant


In [21]:
queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Flushing,Chinese Restaurant,Hotpot Restaurant,Dim Sum Restaurant,Szechuan Restaurant,Cantonese Restaurant,Fried Chicken Joint,Bakery,Dongbei Restaurant,Dumpling Restaurant,Food Stand
56,Hillcrest,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Dim Sum Restaurant,Vegetarian / Vegan Restaurant,Halal Restaurant,Bakery,Buffet,Caribbean Restaurant,Coffee Shop


In [22]:
queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Woodside,Chinese Restaurant,Halal Restaurant,Indian Restaurant,Vietnamese Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant
8,Richmond Hill,Chinese Restaurant,Shanghai Restaurant,Japanese Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop
14,Ridgewood,Chinese Restaurant,Asian Restaurant,Buffet,Cantonese Restaurant,Halal Restaurant,Bakery,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
21,Whitestone,Chinese Restaurant,Asian Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
23,Auburndale,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Halal Restaurant,Bakery,Buffet,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
30,Briarwood,Chinese Restaurant,Asian Restaurant,Indian Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant
36,St. Albans,Chinese Restaurant,Asian Restaurant,Halal Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
54,Jamaica Estates,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Halal Restaurant,Bakery,Buffet,Coffee Shop,Dim Sum Restaurant,Dongbei Restaurant
57,Ravenswood,Chinese Restaurant,Noodle House,Latin American Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant,Coffee Shop
60,Lefrak City,Chinese Restaurant,Taiwanese Restaurant,Shanghai Restaurant,Kosher Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Bakery,Buffet,Cantonese Restaurant,Caribbean Restaurant


In [24]:
queens_venues_chr.shape

(999, 7)

# Discussion and Conlusion

All the result could be works for limited data but not for a large and unstable one. The result reflects that it could be correct or not. Which is unstable. There are a lot of area might be good for the restaurant but the analyst result is not include. Which means it might be not reliable. It need people to double check could be the best way after the data analyse. It can only provide some of position as recommend and the truth should be explore by real people. 


# Result

After the project, We got the conclusion, The cluster 3 will be the best position to have the restaurant
